# Extração da Drogaria Minas Mais

In [38]:
# Import do código
import requests
from bs4 import BeautifulSoup
import os
from concurrent.futures import ThreadPoolExecutor
import time
import json
import pandas as pd
import math

In [39]:
# Capturando todas as URLs

# Função de acessar site
all_workers = os.cpu_count()

def access_site(url, max_attempts=all_workers):
    headers = {'Accept': '*/*', 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    for retry in range(max_attempts):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
        except Exception as e:
                print(f"An error occurred while accessing URL: {url}: {e}. Retrying... (Attempt {retry + 1}/{max_attempts})")
                time.sleep(1)

# Função captura de todos os nomes de marcas e criação de URLs
def getNomeMarcas():
    response = access_site("https://www.drogariasminasmais.com.br/medicamentos")
    soup = BeautifulSoup(response.text, 'html.parser')
    sessaoMarcasGrandes = soup.find('div', class_='vtex-search-result-3-x-filter__container vtex-search-result-3-x-filter__container--responsive-result-content-mz0001 bb b--muted-4 vtex-search-result-3-x-filter__container--brand')
    sessaoMarcas = sessaoMarcasGrandes.find('div', class_='vtex-search-result-3-x-filterTemplateOverflow vtex-search-result-3-x-filterTemplateOverflow--responsive-result-content-mz0001 pb5 overflow-y-auto')
    marcas = sessaoMarcas.find_all('label', class_='vtex-checkbox__label w-100 c-on-base pointer')
    return marcas

# Criando uma lista de URLs ainda não bem estruturadas
urlSuja = getNomeMarcas()
urlLimpa = []

# Limpando lista das URLs
for i in urlSuja:
    # Criando texto separado e minusculo com "-"
    texto = i.text.lower()
    textoNovo = texto.replace(" ", "-")
    textoNovissimo = textoNovo.replace("/", "-")

    # Salvando tudo em uma lista
    urlLimpa.append(textoNovissimo)

listaUrls = []

# Criando todas as URLs
for j in urlLimpa:
    URL = "https://www.drogariasminasmais.com.br/medicamentos/" + j + "?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page="
    listaUrls.append(URL)

In [40]:
# Achando número de produtos
url = "https://www.drogariasminasmais.com.br/medicamentos/hypera?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1"
requisicao = requests.get(url)
soup = BeautifulSoup(requisicao.text, 'html.parser')
numeroProdutos = soup.find_all('script')[-34].text
numeroProdutos = numeroProdutos[:-8].split("quantity")[66:]

listaNumeroProdutos = []
listaNumeroProdutosCorreta = []

for i in numeroProdutos:
    listaNumeroProdutos.append(i.split(",")[0])

for i in listaNumeroProdutos:
    aux = i.replace('"', "")
    aux = aux.replace(":", "")
    listaNumeroProdutosCorreta.append(int(aux))

In [100]:
# Função getAllData
def getAllData(): 
    def getJSON(soup):
        jsonSujo = soup.find_all('script')[-34].text
        jsonSujo = jsonSujo[:-8]
        jsonLimpo = jsonSujo.split("__STATE__ = ")[1]
        return jsonLimpo

    total_pages = []

    for i in listaNumeroProdutosCorreta:
        if i <= 15:
            total_pages.append(2)
        else:
            total_pages.append(math.ceil(i/15) + 1)

    listaJSON = []
    k = 0

    for i in listaUrls:
        for page_number in total_pages[k:]:
            for paginaAtual in range(1, page_number):
                url = str(i) + str(paginaAtual)
                requisicao = requests.get(url)
                soup = BeautifulSoup(requisicao.text, 'html.parser')
                listaJSON.append(getJSON(soup))
            print(k)
            print(page_number)
            print(i)
            print(len(listaJSON))
            print("---------------------------------------")
            k = k + 1
            break
    
    return listaJSON

In [102]:
# Função de execução rápida
all_workers = os.cpu_count()

with ThreadPoolExecutor(max_workers=all_workers) as executor:
    future = executor.submit(getAllData())
    listaJSON = future.result()

0
26
https://www.drogariasminasmais.com.br/medicamentos/hypera?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=
25
---------------------------------------
1
21
https://www.drogariasminasmais.com.br/medicamentos/ache-rx?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=
45
---------------------------------------
2
17
https://www.drogariasminasmais.com.br/medicamentos/eurofarma-rx?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=
61
---------------------------------------


IndexError: list index out of range

In [12]:
# Teste de getDados
def getDados(soup):
    jsonSujo = soup.find_all('script')[-34].text
    jsonSujo = jsonSujo[:-8]
    jsonLimpo = jsonSujo.split("__STATE__ = ")[1]
    return jsonLimpo

total_pages = [25]
listaJSON = []

for page_number in total_pages:
    for paginaAtual in range(1, page_number + 1):
        url = "https://www.drogariasminasmais.com.br/medicamentos/hypera?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=" + str(paginaAtual)
        requisicao = requests.get(url)
        soup = BeautifulSoup(requisicao.text, 'html.parser')
        listaJSON.append(getDados(soup))

In [36]:
# Criando o JSON e capturando todos os códigos de produtos
ean = []
precoComDesconto = []
precoSemDesconto = []
desconto = []
marcas = []
nomes = []

for i in range(0, len(listaJSON)):
    dados = json.loads(listaJSON[i])
    listaProdutosCerta = []
    listaProdutos = []
    j = 0 

    for i in dados.keys():
        if i.split("$ROOT_QUERY")[0] == "":
            break
        if j % 18 == 0:
            listaProdutos.append(i)
        j = j + 1

    for i in listaProdutos:
        listaProdutosCerta.append(i.split(".")[0].replace("$", ""))

    # Pegando todos os dados dos produtos
    for i in listaProdutosCerta:
        nomes.append(dados[i]["productName"])
        marcas.append(dados[i]["brand"])

    for i in listaProdutosCerta:
        # Parte do JSON que tem o EAN e precos
        localJson1 = str(i) + '.items({"filter":"ALL"}).0'
        dadosDesejados1 = dados.get(localJson1)

        ean.append(dadosDesejados1["ean"])

    for i in listaProdutosCerta:
        # Parte do JSON que tem o EAN e precos
        localJson2 = "$" + str(i) + '.items({"filter":"ALL"}).0.sellers.0.commertialOffer'
        dadosDesejados2 = dados.get(localJson2)

        precoComDesconto.append(dadosDesejados2["Price"])
        precoSemDesconto.append(dadosDesejados2["ListPrice"])
        desconto.append((1 - (dadosDesejados2["Price"]/dadosDesejados2["ListPrice"])))

In [32]:
# Criando dataFrame
data = {"nome": nomes, "marcas": marcas, "EAN": ean, "Preço com desconto": precoComDesconto, "Preço sem desconto": precoSemDesconto, "Desconto": desconto}
df = pd.DataFrame(data)

In [33]:
# Exportando tudo para um csv
df.to_excel('DadosMinasMais.xlsx',sheet_name='Sheet1')